<a href="https://colab.research.google.com/github/dhsolutionspak/MSCUI/blob/main/PFAI_Mudassir_Reportipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning for Sentiment Analysis Using BERT Based
Supervised Fine-Tuning

**Mudassir Jaleel**
**FA24/RAI/010**

In [1]:
!pip install -U "tensorflow-text==2.13.*"

## Importing The Libaries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import os
print(os.listdir("/content"))

['.config', 'sample_data']


## Loading The Data

In [3]:
!pip install --quiet tensorflow_text
import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
comments = pd.read_csv('/content/UScomments.csv', on_bad_lines='skip')
US_comments = comments.iloc[:1000]

In [5]:
US_comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


## Let's do some analysis and Data Cleaning on both the datasets.

In [6]:
US_comments.shape

(1000, 4)

In [7]:
US_comments.nunique()

,0
video_id,11
comment_text,991
likes,8
replies,5


In [8]:
US_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1000 non-null   object
 1   comment_text  1000 non-null   object
 2   likes         1000 non-null   object
 3   replies       1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [9]:
US_comments.isnull().sum()

,0
video_id,0
comment_text,0
likes,0
replies,0


In [10]:
US_comments.likes = US_comments.likes.astype(int)
US_comments.replies = US_comments.replies.astype(int)

In [11]:
US_comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [12]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords') # Download the stopwords dataset
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize # Import word_tokenize
from nltk.corpus import stopwords # Import stopwords

def preprocess(text):
    text = re.sub(r'(^|\s)@(\w+)', '', text)
    text = re.sub(r'\bhttps?://\S+\b', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    tokens = word_tokenize(text) # Now word_tokenize is accessible
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    text = ' '.join(tokens)
    return text

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
US_comments['comment_text'] = US_comments['comment_text'].apply(preprocess)

In [14]:
US_comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,logan paul yo big day,4,0
1,XpVt6Z1Gjjo,ive following start vine channel seen vlogs,3,0
2,XpVt6Z1Gjjo,say hi kong maverick,3,0
3,XpVt6Z1Gjjo,fan attendance,3,0
4,XpVt6Z1Gjjo,trending,3,0


## Let's do the Sentiment Analysis on the US Comments Dataset

In [15]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

## Setting The Sentiment Scores

In [17]:
US_comments['Sentiment Scores'] = US_comments['comment_text'].apply(lambda x:sia.polarity_scores(x)['compound'])

In [18]:
US_comments.head()

,video_id,comment_text,likes,replies,Sentiment Scores
0,XpVt6Z1Gjjo,logan paul yo big day,4,0,0.0000
1,XpVt6Z1Gjjo,ive following start vine channel seen vlogs,3,0,0.0000
2,XpVt6Z1Gjjo,say hi kong maverick,3,0,0.0000
3,XpVt6Z1Gjjo,fan attendance,3,0,0.3182
4,XpVt6Z1Gjjo,trending,3,0,0.0000


## Classifying the Sentiment scores as Positive, Negative and Neutral

In [19]:
US_comments['Sentiment'] = US_comments['Sentiment Scores'].apply(lambda s : 'Positive' if s > 0 else ('Neutral' if s == 0 else 'Negative'))

In [20]:
US_comments.head()

,video_id,comment_text,likes,replies,Sentiment Scores,Sentiment
0,XpVt6Z1Gjjo,logan paul yo big day,4,0,0.0000,Neutral
1,XpVt6Z1Gjjo,ive following start vine channel seen vlogs,3,0,0.0000,Neutral
2,XpVt6Z1Gjjo,say hi kong maverick,3,0,0.0000,Neutral
3,XpVt6Z1Gjjo,fan attendance,3,0,0.3182,Positive
4,XpVt6Z1Gjjo,trending,3,0,0.0000,Neutral


In [21]:
US_comments.Sentiment.value_counts()

,count
Sentiment,
Positive,439
Neutral,339
Negative,222


In [22]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [23]:
# Map sentiment values to numerical labels
sentiment_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2}
US_comments['sentiment_label'] = US_comments['Sentiment'].map(sentiment_mapping)  # Assuming 'Sentiment' is your target column

# X and y are derived from the DataFrame
X = US_comments['comment_text']
y = US_comments['sentiment_label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
X_train

,comment_text
29,love much love meet live israel really inspire...
535,trump literally didnt know fredrick douglas re...
695,amazing
557,make usa great take billion cut hud budget wea...
836,hold fuvk netflix included
...,...
106,dont sorry pewds everybody say something bad e...
270,absolutely beautiful wtf apple dollar xpensive
860,new movie next
435,b b nnwhere park flying mercedes


In [25]:
US_comments

,video_id,comment_text,likes,replies,Sentiment Scores,Sentiment,sentiment_label
0,XpVt6Z1Gjjo,logan paul yo big day,4,0,0.0000,Neutral,2
1,XpVt6Z1Gjjo,ive following start vine channel seen vlogs,3,0,0.0000,Neutral,2
2,XpVt6Z1Gjjo,say hi kong maverick,3,0,0.0000,Neutral,2
3,XpVt6Z1Gjjo,fan attendance,3,0,0.3182,Positive,0
4,XpVt6Z1Gjjo,trending,3,0,0.0000,Neutral,2
...,...,...,...,...,...,...,...
995,Ayb_2qbZHm4,generic attention comment,0,0,0.0000,Neutral,2
996,Ayb_2qbZHm4,zero amount funny,0,0,0.4404,Positive,0
997,Ayb_2qbZHm4,honest see random color hairgender study lesbi...,0,0,0.7351,Positive,0
998,Ayb_2qbZHm4,joke cant apply going call woman woman offensi...,0,0,-0.8074,Negative,1


In [26]:
# Define BERT model for fine-tuning
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4", trainable=True)

# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)

# Pooled output
pooled_output = outputs["pooled_output"]

# Dropout and classification layers
drop_out = tf.keras.layers.Dropout(0.3, name='dropout')(pooled_output)  # Increased dropout for regularization
output = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(drop_out)  # 3 classes for multi-class

# Build the model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

In [27]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [28]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
batch_size = 16
epochs = 5
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)


Epoch 1/5
40/40 [==============================] - 1169s 28s/step - loss: 1.0999 - accuracy: 0.4531 - val_loss: 0.9143 - val_accuracy: 0.5375
Epoch 2/5
40/40 [==============================] - 1097s 27s/step - loss: 0.7289 - accuracy: 0.6906 - val_loss: 0.7851 - val_accuracy: 0.6625
Epoch 3/5
40/40 [==============================] - 1112s 28s/step - loss: 0.4228 - accuracy: 0.8438 - val_loss: 0.6731 - val_accuracy: 0.7125
Epoch 4/5
40/40 [==============================] - 1102s 28s/step - loss: 0.2305 - accuracy: 0.9187 - val_loss: 0.7767 - val_accuracy: 0.7437


In [29]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Test Loss: 0.7697
Test Accuracy: 0.6550


In [30]:
# Save the fine-tuned model
model.save('fine_tuned_bert_keras')

**Result and Discussion**

In the paper the accuracy was averyage 85%+ for all datasets , here i used the youtube comments datasets which is one of them and i got 65% . Its low due to i just took 1st 1000 Rows , there were 6Lac + Rows in the dataset . which will provide us a good accuaracy after training the data on them . I used 1000 because for that we need a big machine to process !